In [11]:
#Importing the required libraries
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [12]:
#Downloading the training and testing data
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [13]:
batch_size = 64

#Creating data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X[N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X[N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [14]:
#Getting the cpu or gpu device for training
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"The model is using {device} device")

#Defining the model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)
print(model)

The model is using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [15]:
#Creating the optimizer and loss function
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)

In [16]:
#Defining training and testing functions
def train(dataloader, model, loss_fn, optimzer):
    size = len(dataloader.dataset)
    model.train()

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        #Compute the prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        #Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch+1)*len(X)
            print(f"Loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()

    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f" Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")

In [17]:
#Training the model
epochs = 50

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

print("Done!")

Epoch 1
-------------------------------
Loss: 2.302865 [   64/60000]
Loss: 0.301382 [ 6464/60000]
Loss: 0.361357 [12864/60000]
Loss: 0.430134 [19264/60000]
Loss: 0.178793 [25664/60000]
Loss: 0.341149 [32064/60000]
Loss: 0.241165 [38464/60000]
Loss: 0.233154 [44864/60000]
Loss: 0.286598 [51264/60000]
Loss: 0.302745 [57664/60000]
 Test Error: 
 Accuracy: 91.6%, Avg loss: 0.273742

Epoch 2
-------------------------------
Loss: 0.108197 [   64/60000]
Loss: 0.267546 [ 6464/60000]
Loss: 0.235840 [12864/60000]
Loss: 0.215688 [19264/60000]
Loss: 0.221602 [25664/60000]
Loss: 0.301334 [32064/60000]
Loss: 0.163739 [38464/60000]
Loss: 0.093933 [44864/60000]
Loss: 0.277198 [51264/60000]
Loss: 0.351567 [57664/60000]
 Test Error: 
 Accuracy: 89.6%, Avg loss: 0.397730

Epoch 3
-------------------------------
Loss: 0.272929 [   64/60000]
Loss: 0.133575 [ 6464/60000]
Loss: 0.252845 [12864/60000]
Loss: 0.245125 [19264/60000]
Loss: 0.131587 [25664/60000]
Loss: 0.208961 [32064/60000]
Loss: 0.188031 [38464/

In [18]:
#Saving the model
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth
